In [2]:
from pathlib import Path
import os
## Creating a language chain that can help take a github repo and other documentation and create a language chain that can reference those documents when needed.
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, 
    Language,
)
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import openai
load_dotenv()

# Get OpenAI and Pinecone API keys
openai.api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_KEY")
openai.organization = os.getenv("OPENAI_ORG")
pinecone_env = os.getenv("PINECONE_ENV")

# Initialize Pinecone
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)





In [50]:
from git import Repo

url = "https://github.com/openai/openai-python"
repo = Repo.clone_from(url, to_path=f"./example_data/test_repo1")
branch = repo.head.reference
print(branch.name)


main


In [51]:
root_dir = './example_data/test_repo1/'


docs = []

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size = 1000, chunk_overlap = 15
)
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith(".py") and "/.venv/" not in dirpath:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                docs.extend(loader.load_and_split(text_splitter=python_splitter))
            except Exception as e:
                pass
print(f"{len(docs)}")

361


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=15
    )

In [5]:
# Create the embeddings object
embeddings = OpenAIEmbeddings(
    openai_api_key = openai.api_key,
    openai_organization= openai.organization
)

In [6]:
from langchain.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader(
    file_path="./bakespace_api.pdf",
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 15
)

In [7]:
pages = pdf_loader.load_and_split(text_splitter=text_splitter)

In [8]:
docs = pages

In [9]:
# Connect to the Pinecone vector store
index = pinecone.GRPCIndex('dave-code')

In [10]:
for i, doc in enumerate(docs):
    doc.metadata["id"] = f'Bakespace_API_{i}'
    doc.metadata['type'] = 'PDF'
    doc.metadata['repo'] = 'Bakespace_API'



In [11]:
texts = []
for doc in docs:
    texts.append(doc.page_content)

In [12]:
vectors=embeddings.embed_documents(texts)

In [13]:
len(vectors), len(vectors[0])

(39, 1536)

In [14]:
# Create a new dataframe from the doc data
import pandas as pd

# Set the columns to "text", "id", "metadata"
df = pd.DataFrame(columns=["values", "id", "metadata"])

# Loop through the docs and set the values for each column
for i, doc in enumerate(docs):
    df = df.append(
        {
            "values": vectors[i],
            "id": doc.metadata["id"],
            "metadata": doc.metadata,
        },
        ignore_index=True,
    )

C:\Users\sjufa\AppData\Local\Temp\ipykernel_19596\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_19596\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_19596\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_19596\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\sjufa\AppData\Local\Temp\ipykernel_19596\464683487.py:9: FutureWarning: The frame.append method is deprecated and will be r

In [15]:
# Create a dictionary of the dataframe
data = df.to_dict(orient="records")

# Set the value of the text key to the texts list
for i, doc in enumerate(data):
    doc["metadata"]['text'] = texts[i]

    

In [16]:
# Upsert the data to the Pinecone vector store
index.upsert(vectors=data, batch_size=50)

Upserted vectors:   0%|          | 0/39 [00:00<?, ?it/s]

upserted_count: 39

In [11]:
index.upsert_from_dataframe(df, batch_size=50)

sending upsert requests:   0%|          | 0/5910 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/119 [00:00<?, ?it/s]

upserted_count: 5910

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 5910}},
 'total_vector_count': 5910}

In [24]:
vectorstore = Pinecone.from_existing_index(index_name="dave-code", embedding = embeddings, text_key="text")

In [25]:
query = "what does the generate_cocktail function do?"

vectorstore.similarity_search(query, k=5)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


[Document(page_content='entities = generation_resp["entities"]\n                self.mlflg.html(dependency_tree, "dep-" + hash_string(generation.text))\n                self.mlflg.html(entities, "ent-" + hash_string(generation.text))', metadata={'id': 'Langchain_752', 'repo': 'Langchain', 'source': './langchain/test_repo1/langchain\\callbacks\\mlflow_callback.py', 'type': 'Github Repo'}),
 Document(page_content='return ""', metadata={'id': 'Langchain_1981', 'repo': 'Langchain', 'source': './langchain/test_repo1/langchain\\document_loaders\\notebook.py', 'type': 'Github Repo'}),
 Document(page_content='}\n        )', metadata={'id': 'Langchain_4654', 'repo': 'Langchain', 'source': './langchain/test_repo1/langchain\\vectorstores\\vectara.py', 'type': 'Github Repo'}),
 Document(page_content='`argilla.apikey` will be used.', metadata={'id': 'Langchain_582', 'repo': 'Langchain', 'source': './langchain/test_repo1/langchain\\callbacks\\argilla_callback.py', 'type': 'Github Repo'}),
 Document(